In [47]:
import pandas as pd

import sqlite3
import csv

In [48]:
cbb = pd.read_csv('cbb.csv')

In [49]:
# replacing the postseason with an easy to read solution
# the number refers to what round each team reached

final_round = {
    'R64' : 1,
    'R32' : 2,
    'S16' : 3,
    'E8'  : 4,
    'F4'  : 5,
    '2ND' : 6,
    'Champions' : 7,
    None : 0,
    'R68' : 0
}

cbb['POSTSEASON'] = cbb['POSTSEASON'].replace(final_round)

In [50]:
# renaming columns for sql purposes

cbb.columns = ['TEAM', 'CONF', 'G', 'W', 'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D',
       'TOR', 'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', 'Two_P_O', 'Two_P_D', 'Three_P_O',
       'Three_P_D', 'ADJ_T', 'WAB', 'POSTSEASON', 'SEED', 'YEAR']

In [51]:
cbb

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,FTRD,Two_P_O,Two_P_D,Three_P_O,Three_P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
0,North Carolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,...,30.4,53.9,44.6,32.7,36.2,71.7,8.6,6,1.0,2016
1,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,...,22.4,54.8,44.7,36.5,37.5,59.3,11.3,6,1.0,2015
2,Michigan,B10,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,...,30.0,54.7,46.8,35.2,33.2,65.9,6.9,6,3.0,2018
3,Texas Tech,B12,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,...,36.6,52.8,41.9,36.5,29.7,67.5,7.0,6,3.0,2019
4,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,...,26.9,56.3,40.0,38.2,29.0,71.5,7.7,6,1.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450,Michigan St.,B10,35,26,111.4,87.8,0.9392,50.6,44.5,20.8,...,32.4,50.4,44.3,34.1,30.1,64.4,6.7,3,3.0,2013
2451,Arizona,P12,35,27,114.4,92.2,0.9229,52.5,46.6,19.5,...,32.9,50.6,43.4,37.1,35.8,66.8,4.6,3,6.0,2013
2452,Oregon,P12,37,28,104.8,88.6,0.8728,49.3,46.4,21.4,...,33.3,49.1,44.9,33.3,33.4,69.2,2.9,3,12.0,2013
2453,La Salle,A10,34,24,112.0,96.2,0.8516,51.9,49.3,17.1,...,28.5,49.3,50.6,37.7,30.2,66.0,0.3,3,13.0,2013


In [52]:
con = sqlite3.connect('cbb.db')

cur = con.cursor()

In [53]:
cbb.to_sql('cbb', con, if_exists = 'replace', index = False)

In [54]:
cur.execute("""SELECT DISTINCT postseason
               FROM cbb""").fetchall()

[(6,), (7,), (4,), (5,), (0,), (2,), (1,), (3,)]

In [55]:
cbb_s16 = pd.read_sql("""SELECT * 
                         FROM cbb 
                         WHERE postseason >= 3""", con)

In [56]:
Avg_by_round = pd.read_sql("""SELECT AVG(adjoe), AVG(adjde), AVG(barthag), AVG(efg_o), AVG(efg_d), AVG(tor), 
                      AVG(tord), AVG(orb), AVG(drb), AVG(ftr), AVG(ftrd), AVG(two_p_o), 
                      AVG(two_p_d), AVG(three_p_o), AVG(three_p_d), AVG(adj_t), AVG(wab), postseason
               FROM cbb
               WHERE postseason >= 1
               GROUP BY postseason""", con)

In [57]:
cbb_s16.to_csv('cbb_s16.csv')

In [58]:
Avg_by_round.to_csv('avg_by_round.csv')